<font size="5">FORECASTING MEDIAS MOVILES</font>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<font size="4">INFORMACION</font>

In [ ]:
df = pd.read_excel('DATOS.xlsx')
df.info()

In [ ]:
#Graficar
plt.figure(figsize = (12, 6))
plt.plot(df['TRANS DATE'], (df['ORIGINAL GROSS AMT']))

plt.xlabel("Periodos", fontsize = 14)
plt.ylabel("Gastos", fontsize = 14)
plt.title("Serie de Tiempo", fontsize = 18)
plt.show()

In [ ]:
df['month_day'] = df['TRANS DATE'].apply(lambda x:pd.Timestamp(x).strftime('%Y-%m-%d'))
variables = pd.concat([df['TRANS DATE'], df['ORIGINAL GROSS AMT'].round(0).astype(int), df['month_day']],  axis = 1)
variables

<font size="4">TRATAMIENTO DE DATOS</font>

In [ ]:
#Datos Diarios
diario = variables.groupby(by=['month_day']).sum().reset_index()
diario.head()

In [ ]:
#Graficar
plt.figure(figsize = (12, 6))
plt.plot(diario.index, (np.log(diario['ORIGINAL GROSS AMT'])))

plt.xlabel("Periodos", fontsize = 14)
plt.ylabel("Gastos", fontsize = 14)
plt.title("Serie de Tiempo", fontsize = 18)
plt.show()

<font size="4">MODELO ARIMA</font>

In [ ]:
from statsmodels.tsa.stattools import adfuller
adf = adfuller(np.log(diario['ORIGINAL GROSS AMT']).fillna(method='ffill'))
print('El test es: ',adf[0])
print('El P value es: ', adf[1])
print('Valores Criticos: ', adf[4])

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")
stepwise_fit = auto_arima(np.log(diario['ORIGINAL GROSS AMT']).fillna(method='ffill'), trace = True, supress_warnings = True)

In [ ]:
train = diario.iloc[:-30]
test = diario.iloc[-30:]

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
mod = ARIMA((np.log(train['ORIGINAL GROSS AMT'])).fillna(method='ffill'), order = (5,0,2))
model = mod.fit()
model.summary()

<font size="4">PRONOSTICO</font>

In [ ]:
start = len(train)
end = len(train) + len(test)-1
pred = model.predict(start=start, end=end, typ='levels')
print(pred)

In [ ]:
pred.plot(legend = True)
np.log(test['ORIGINAL GROSS AMT']).plot(legend = True)